In [1]:
import numpy as np
from tqdm import tqdm
from constants import MAX_SEQ_LEN

with open('../hw_train.csv', 'rb') as f:
    ls = f.readlines()[1:]
# count vocabulary and max sequence length
lyrics_map = {}
bos_token = 's'.decode('utf8')
eos_token = 'e'.decode('utf8')
line_token = 'l'.decode('utf8')
oov_token = 'o'.decode('utf8')

none_lyrics_words = [u'編詞', u'作曲', u'作詞', u'編曲', u'監製']
vocab = [bos_token, eos_token, line_token]
lens = []
c = 0 
with tqdm(total=len(ls)) as pbar:
    for l in ls:
        l = l.decode('utf8')
        no, l = l.strip().split(',')
        no = int(no)
        if no not in lyrics_map:
            lyrics_map[no] = []
        lyrics_map[no].append(l)
        pbar.update(1)
# split into Q,K pair
with tqdm(total=len(lyrics_map)) as pbar:
    for k,v in lyrics_map.items():
        line_num = len(v)
        V = [bos_token, ]
        Q = [bos_token, ]
        for i,l in enumerate(v):
            exist_none_layrics_word = False
            for nw in none_lyrics_words:
                if nw in l:
                    exist_none_layrics_word = True
                    break
            if exist_none_layrics_word:
                continue
            
            for ws in l.strip().split(' '):
                for w in ws:
                    if i <= line_num//2:
                        V.append(w)
                    else:
                        Q.append(w)
                    vocab.append(w)
            if i <= line_num//2:
                V.append(line_token)
            else:
                Q.append(line_token)
        V.append(eos_token)
        Q.append(eos_token)
        lens.append(len(Q))
        lens.append(len(V))
        lyrics_map[k] = (Q, V)
        pbar.update(1)
            
print 'vocabulary size : %d, max len : %.0f, mean : %.0f, std : %.0f' % (len(set(vocab)), np.max(lens), np.mean(lens), np.std(lens))
print 'lyrics number : %d' % len(lyrics_map)
# print np.max(lens), np.mean(lens), np.std(lens)



100%|██████████| 18338/18338 [00:02<00:00, 7511.94it/s]


vocabulary size : 6456, max len : 4026, mean : 179, std : 118
lyrics number : 18338


In [2]:
import numpy as np
import keras
import pickle
from constants import MAX_SEQ_LEN, VOCAB_DIM
from keras.preprocessing.sequence import pad_sequences

texts = vocab

tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_DIM, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=False, split=' ', char_level=True, oov_token=oov_token)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

with open('tokenizer','wb') as f:
    pickle.dump(tokenizer, f)


Using TensorFlow backend.


Found 6457 unique tokens.


In [3]:
from constants import MAX_SEQ_LEN
from keras.preprocessing.sequence import pad_sequences

idx = np.random.permutation(len(lyrics_map))
val_num = len(lyrics_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[val_num:]}
val_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[:val_num]}
def boostrap_generator(lyrics_map, max_seq_len, tokenizer):
    while True:
        keys = lyrics_map.keys()
        for idx in np.random.permutation(len(lyrics_map)):
            no = keys[idx]
            Q, K = lyrics_map[no]
            Q = np.array([tokenizer.word_index[w] if w in tokenizer.word_index else tokenizer.word_index[oov_token] for w in Q], dtype=np.int32).reshape(1,-1)
            K = np.array([tokenizer.word_index[w] if w in tokenizer.word_index else tokenizer.word_index[oov_token] for w in K], dtype=np.int32).reshape(1,-1)
            yield Q, K
    
bat = 4
G = boostrap_generator(train_u_map, MAX_SEQ_LEN, tokenizer)
q, k = next(G)
    
print q.shape, k.shape


(1, 108) (1, 116)


### RNN

In [4]:
import torch
from Seq2seq import Seq2seq
from constants import D_MODEL, MAX_SEQ_LEN

bat = 7
lay_num = 2
model = Seq2seq(D_MODEL, lay_num).cuda()
# print(o.size())
q, k = next(G)
q = torch.LongTensor(q).cuda()
k = torch.LongTensor(k).cuda()
# Q = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()
# K = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()

o = model(q, k)
print(o.size())
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/backends/cudnn/__init__.py:102: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


torch.Size([1, 98, 6451])
2054195


In [5]:
import torch.nn as nn
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%s<split>%s<split>%s<split>%s\n' % (n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label)
    log_file_stream.write(log_text.encode('utf8'))
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    pred = torch.argmax(pred, dim=-1)
    mask = torch.ones_like(label, dtype=torch.uint8)
#     
    acc = pred == label
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
    
    return acc
def seq2text(output, index_word):
    assert len(output.shape) == 1
    seq = output
    s = ''
    for i in seq:
        i = int(i.item())
        if i == 0:
            continue
        if i not in index_word:
            w = oov_token
        else:
            w = index_word[i]
        
        s += w
    return s
    
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=100)
loss_q = deque(maxlen=10)

val_acc_q = deque(maxlen=100)
val_loss_q = deque(maxlen=10)

t = time.time()
best_loss = float('inf')

epochs = 100
batch_size = 1
G = boostrap_generator(train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = boostrap_generator(val_u_map, MAX_SEQ_LEN, tokenizer)
# criterion = nn.CrossEntropyLoss(reduction='none')
weight = torch.ones([VOCAB_DIM+1,], dtype=torch.float)
weight[tokenizer.word_index[line_token]] = 100.

# criterion = nn.CrossEntropyLoss(weight=weight.cuda())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log.txt', 'w') as f:
    with open('best.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                q, k = next(G)
                q = torch.LongTensor(q).cuda()
                k = torch.LongTensor(k).cuda()

                q.requires_grad_(False)
                k.requires_grad_(False)


                output = model(q, k)
                y = q[:,1:]
#                 loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) 
                loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                label = y

                pred = output[:,:-1,:,]
                acc = normal_acc(pred, label)
                acc_q.append(acc)
                train_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                train_label= seq2text(k[0,:], tokenizer.index_word)
                loss.backward()
                loss_q.append(loss.item())
                
                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    q, k = next(val_G)
                    a = q
                    b = k
                    q = torch.LongTensor(q).cuda()
                    k = torch.LongTensor(k).cuda()
                    
                    q.requires_grad_(False)
                    k.requires_grad_(False)
                    
                    output = model(q, k)
                    y = q[:,1:]
                    val_loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                    label = y

                    pred = output[:,:-1,:,]
                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                    val_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                    val_label= seq2text(k[0,:], tokenizer.index_word)
                    val_loss_q.append(val_loss.item())


                
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)
                loss = np.mean(acc_q)
                val_loss = np.mean(val_acc_q)
                
#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item(), val_loss.item()), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, f,'./tmp.pt')
                if val_loss.item() < best_loss and it > 100:
                    torch.save(model, './best.pt')
                    best_loss = val_loss
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_loss))
                    best_log.flush()
                if it % 2000 == 0 and it >= 100: 
                    print 'train pred : %s\ntrain label : %s' % (train_pred, train_label)
                    print 'validation pred : %s\nvalidation label : %s' % (val_pred, val_label)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/100000000 [00:00<?, ?it/s]

start training.


  0%|          | 2003/100000000 [04:25<3104:42:26,  8.95it/s, acc : 0.202, val_acc : 0.190, loss : 0.202, val_loss : 0.190]

train pred : 我你么l不以l我是我我我的lll我要要一我lll我要要一你lll我弃我我l有我ll我天的的的我个l我间的有你ll我有我的要llll我们的不有你ll我我我心要我ll我爱我寞l我我ll我天的的我我我l我我么l不以l我是我我我llll我要要我我lll我要要一你lll我弃我ll有我ll我天的的的我个l我间的有你ll我有我的要llll我们的不有你ll我我我心要我ll我爱我寞l我我ll我寞
train label : s首先告诉你我是一个人l然后再告诉你我刚单身l孤单的夜里遭遇伤心l摇摇手机排遣郁闷l如果你现在也是一个人l感谢你接受我添加邀请l都是陌生人不要拘谨l反正神马都是浮云l今夜我愿陪着你l说什么都可以l只要别跟我谈感情l不想再看旧伤痕l不想再为谁冲动l放下过去只要开心l今夜我们在一起l时空没有距离l只是我不想谈感情l我们都没有情人l当然更不是情人l别把寂寞当做缘分l如果你现在也是一个人l感谢你接受我添加邀请l都是陌生人不要拘谨l反正神马都是浮云l今夜我愿陪着你le
validation pred : 我们我个人雨lll眼ll我们我个人些的里的爱光l我们我个人有人的ll人ll我们我个人么l天的人光l我们我个人的l的l人光l我空的亮我里个l我界的必间是不ll我是要我的扎的我实我的要我个人llll我以够们我人狂的我们要我一的爱ll我然我我的天l我实我的是我个人llll我寞
validation label : s我是一个天真活泼的孩子l我是一个喜欢春天的孩子l我是一个爱笑的孩子l我是一个没有玩具的孩子l我是一个有理想的孩子l我是一个没有人疼的孩子l天会阴到哪一天l世界何时才会改变l只想努力挣钱l其实我不是一个文艺青年l可能我是个疯子l我不愿做你的棋子l虽然年过三十l其实我还是一个文艺青年l我是一个内心浮躁的孩子le


  0%|          | 4002/100000000 [08:48<4069:24:47,  6.83it/s, acc : 0.233, val_acc : 0.219, loss : 0.233, val_loss : 0.219] 

train pred : 你一l啦都不l了我是你法的乐ll信l你到后的束ll间llll膀我梭的梭你法ll你我西我ll待l我是爱中们有远l得ll你为你的否的心ll你是我春的你你的你你是我你l是你l你ll你生的界的在经不见ll你一惯的你我手lle是爱天lllll你能道福的否llll你待你一了爱间le你你的你我们道l你为你们有ll见l你llll我寞
train label : s有一日咱若老l找无人甲咱有孝l我会陪你坐惦椅寮l听你讲少年的时阵你有外l吃好吃丑无计较l怨天怨地嘛袂晓l你的手我会甲你牵条条l因为你是我的家后l你将青春嫁置阮兜l你对少年跟阮跟甲老l人情世事已经看透透l有啥人比你卡重要l你的一生献乎阮兜l才知幸福是吵吵闹闹l等待返去的时阵若到l我会让你先走l因为我会呒甘放你为我目屎流l有一日咱若老le
validation pred : 你你你泪的眼水ll我长的的的l来l子ll你爱的你的说你的泣l你方的我l然不阳ll我你你着的l你的福l你然是着我吹然不一我l你们你中中间的的我的我有你的爱子ll你们不多你惜l己l你有你们心月ll你的我护l的己l你是你的方l来ll你们不轻l方l己l你要你我一间见道你方l我们你的的有你我天l我是你我一的见道你方l我们你的的有你我天l我水
validation label : s轻轻的我将离开你l请将眼角的泪拭去l漫漫长夜里未来日子里l亲爱的你别为我哭泣l前方的路虽然太凄迷l请在笑容里为我祝福l虽然迎著风虽然下著雨l我在风雨之中念著你l没有你的日子里l我会更加珍惜自己l没有我的岁月里l你要保重你自己l你问我何时归故里l我也轻声地问自己l不是在此时不知在何时l我想大约会是在冬季l不是在此时不知在何时l我想大约会是在冬季l轻轻的我将离开你le


  0%|          | 4151/100000000 [09:08<4055:46:54,  6.85it/s, acc : 0.214, val_acc : 0.226, loss : 0.214, val_loss : 0.226]


RuntimeError: cublas runtime error : the GPU program failed to execute at /home/k123/git/pytorch/aten/src/THC/THCBlas.cu:258

In [ ]:
print q.shape, k.shape
# print a
# print b
print a.shape
print seq2text(a.flatten(), tokenizer.index_word)
print seq2text(b.flatten(), tokenizer.index_word)
m = model
m = m.cuda()
